## Toy Example for ML Backend with Zero-Shot UIE model (Relation Extraction/Information Extraction)

### 想法：

建立一個ML模型，使模型的預測結果能夠在Label Studio上呈現，也就是讓模型自動標記，人類的工作就是後續校正模型的標記結果。因此在此階段**不會使用到互動學習的概念**，只會單純將模型的預測標記上去。

因此首先需要知道Label Studio的標記格式，接著將模型的Output調整成Label Studio所需的格式，最後透過Label Studio內建處理好的環境，就可以將調整完Label Studio所需的格式的Output貼回Label Studio上。

所以以下會先使用Label Studio人工標記一篇文本，接著透過標記結果判斷Label Studio的標記格式，之後建立ML Backend，把文本丟給Backend中的模型，將模型output對齊Label Studio的標記格式，最後Output結果就結束了。

### 範例任務描述

在此範例中，我們希望讓「RE」模型在一篇判決書中自己標註「精神慰撫金」，並能透過「Relation」正確對應到提出金額的原告，因此在任務中我們需要做以下標記：

1. 對「原告」標記成「原告」的Label。
2. 對「法院判決的精神慰撫金」標記成「法院得心證判斷之適當裁定精神慰撫金」。
3. 對「原告提出的精神慰撫金」標記成「非財產上之損害的求償精神慰撫金」。
4. 對「非2.且非3.的精神慰撫金」標記成「其他精神慰撫金」。
5. 將所有2.和3.的精神慰撫金，透過Relation，連到對應提出金額的原告，類別等同2.或3.的類別。
（詳細在「1. 建立Label Studio環境」將有範例）

### 介紹流程：

1. 建立Label Studio環境, 標記文本（此步驟僅為了示範如何觀察Label Studio標記格式，若以熟悉Label Studio可跳過）。
2. 建立ML Backend。
3. 測試結果。


## 1. 建立Label Studio環境

### 1.1 Create Label Studio Environment

參考[Label Studio官方說明](https://labelstud.io/guide/get_started.html#Quick-start)即可。

![](https://i.imgur.com/KYRPZ2y.png)


### 1.2 Upload Data

接著將 [data](https://github.com/kiangkiangkiang/NLLP/blob/main/ML_Backend_with_LabelStudio/example_data/toy_example.txt) Import至Label Studio

### 1.3 Label Schema Setting

點右上角Setting -> Labeling Interface -> Browse Templates -> Natural Language Processing -> Relation Extraction -> Code

將以下程式貼到Code內 -> Save：
``` javascript
<View>
    <Relations>
        <Relation value="非財產上之損害的求償精神慰撫金"/>
        <Relation value="法院得心證判斷之適當裁定精神慰撫金"/>
        <Relation value="其他精神慰撫金"/>
    </Relations>
    <Labels name="label" toName="text">
        <Label value="原告" background="#44ff00"/>
        <Label value="非財產上之損害的求償精神慰撫金" background="#0062ff"/>
        <Label value="法院得心證判斷之適當裁定精神慰撫金" background="#9e06e5"/>
        <Label value="其他精神慰撫金" background="#ff0000"/>
    </Labels>

    <Text name="text" value="$text"/>
</View>abels>
```

### 1.4 人工標記以觀察Label Studio格式

Label Studio可接受的Import格式請參考[這裡](https://labelstud.io/guide/tasks.html#Basic-Label-Studio-JSON-format)。

最土法煉鋼的方式，就是直接手動標完一篇，看看Label Studio怎麼標文本，在向他對齊就好。

因此根據「範例任務描述」所提到的標記方式，將第一篇標完如下：

![](https://i.imgur.com/uJjIXbo.png)

點選Submit後，透過「Show task source」便可以看到Label Studio標記完的格式：

![](https://i.imgur.com/7h0CYT0.png)

在「Show task source」的json檔內，result後的格式便是後續ML Backend模型output要對齊的格式。

## 2. 建立ML Backend

In [ ]:
!pip install label_studio_ml

### 2.1 Init 一個空的Backend Template

In [6]:
!label-studio-ml init toy_example_for_ml_backend --script simple_ml_backend_template.py
#toy_example_for_ml_backend可改成自己要的名字

Congratulations! ML Backend has been successfully initialized in ./toy_example_for_ml_backend
Now start it by using:
label-studio-ml start ./toy_example_for_ml_backend


### 2.2 寫入ML Backend核心演算法

在simple_ml_backend_template.py中，預設兩個函數：
1. predict(): 模型核心演算法所在位置，當打開Label Studio時，模型會自動把所有「未預測」的資料，丟給predict，餵給參數tasks，因此可以把tasks內的資料取出丟給模型，再將最後結果對齊好return回去，此時如果return格式沒問題，Label Studio便會自動依照return結果做標記。
2. fit(): **本篇不會用到**，標記好資料後，點選Submit或Update，則模型自動會把標記好的結果丟給fit，此時若要動態學習，便可以在這邊實作fine-tune的演算法，但個人覺得一篇learn一次或是自己寫算法讓模型多篇learn一次，都太花時間，因為learn也需要時間，因此實作上有點卡。
   
詳細可以參考[這裡](https://labelstud.io/tutorials/dummy_model.html)。

在此我們將演算法寫進./toy_example_for_ml_backend/simple_ml_backend_template.py內

In [ ]:
#將此block複製到./toy_example_for_ml_backend/simple_ml_backend_template.py內即可
from label_studio_ml.model import LabelStudioMLBase
from paddlenlp import Taskflow
import numpy as np

def postprocessing(predict_result, all_relations):
    predict_result = sorted(predict_result, key=lambda x: x['value']['start'])

    #Delete Overlap Entity
    index = 0
    while index < len(predict_result) - 1:
        if predict_result[index]['value']['start'] == predict_result[index + 1]['value']['start'] :
            if predict_result[index]['value']['score'] > predict_result[index + 1]['value']['score']:
                predict_result.pop(index + 1)
            else:
                predict_result.pop(index)
        else:
            index += 1

    #Delete Repeat Person
    unique_name_map, result_index = [], []
    for i, item in enumerate(predict_result):
        if item['value']['labels'][0] == '原告':
            if item['value']['text'] not in unique_name_map:
                unique_name_map.append(item['value']['text'])
                result_index.append(i)
        else:
            result_index.append(i)
    predict_result = list(np.array(predict_result)[result_index])

    #Add Relation
    start_id_mapping = {predict_result[index]['value']['start']: predict_result[index]['id'] for index in range(len(predict_result))}
    for relation in all_relations:
        people_start = relation['start']
        for relation_type in relation['relations']:
            money_start = relation['relations'][relation_type][0]['start']
            if start_id_mapping.get(people_start) and start_id_mapping.get(money_start):
                predict_result.append({
                    'from_id': start_id_mapping[people_start],
                    'to_id': start_id_mapping[money_start],
                    "type": "relation",
                    "direction": "right",
                    "labels": [
                        relation_type
                    ]
                })
    return predict_result

class SimpleMLBackend(LabelStudioMLBase):
    #載入模型
    def __init__(self, **kwargs) -> None:
        super(SimpleMLBackend, self).__init__(**kwargs)
        my_toy_schema = ['非財產上之損害的求償精神慰撫金', '法院得心證判斷之適當裁定精神慰撫金',
                        {'原告': '非財產上之損害的求償精神慰撫金',
                         '原告': '法院得心證判斷之適當裁定精神慰撫金'}]
        self.model = Taskflow("information_extraction", schema=my_toy_schema)

        #RE任務需用ID連接，因此每個Entitu都定義ID
        self.entity_id = 0
    

    #Model Inference
    def predict(self, tasks, **kwargs):
        print("Start Label...")
        predictions = []
        for task in tasks:
            #task 格式同「Show task source」
            uie_output = self.model(task['data']['text'])[0]
            result, relations = [], []
            for key in uie_output:
                for item in uie_output[key]:
                    #result對齊Label Studio標好的格式                    
                    result.append({
                        'value': {
                            'start': item['start'],
                            'end': item['end'],
                            'score': item['probability'],
                            'text': item['text'],
                            'labels': [key]
                        },
                        'id': str(self.entity_id),
                        'from_name': 'label', 
                        'to_name': 'text',
                        'type': 'labels'
                    })
                    self.entity_id += 1

                    if item.get('relations') is not None:
                        relations.append(item)

            result = postprocessing(predict_result=result, all_relations=relations)
            predictions.append({
                'result': result,
                'model_version': 'my_toy_example_using_uie'
            })

        print("End Label...")
        return predictions


    #NOT IMPLEMENT FOR THIS CASE
    def fit(self, annotation, workdir=None, **kwargs):
        return

## 3. 測試結果

In [ ]:
!label-studio-ml start toy_example_for_ml_backend

接著：
1. 開啟Label Studio -> Settings -> Machine Learning -> Add Model
2. 勾選Retrieve predictions when loading a task automatically
3. 把剛剛在terminal start的backend的IP貼上去
4. 勾選Use for interactive preannotations -> Validate and Save

成功如下：
![](https://i.imgur.com/zVv7bsO.png)

最後回到Label Studio，點選文章即開始標註（可透過Start Backend的Terminal觀察）

Note: 此範例是Zero-Shot的結果，若Fine-tune表現會好很多，詳細請參考PaddleNLP UIE。